In [1]:
from baggingPU import BaggingClassifierPU
import numpy as np
import pandas as pd
from sklearn.svm import SVC

In [2]:
data = pd.read_csv("../data/features/STAD_PCA_5_threshold0.0005.csv",index_col=0)

In [3]:
STAD_high = list(pd.read_csv("../data/grouping/STAD_MSIH_TILsH_TMBH.txt",sep="\t",index_col=0).index)
STAD_EBV = list(pd.read_csv("../data/grouping/STAD_EBV_27.txt",sep="\t",index_col=0).index)
STAD_MSS= list(pd.read_csv("../data/grouping/STAD_MSS_257.txt",sep="\t",index_col=0).index)

In [4]:
R = ['ERR2497990',
 'ERR2497993',
 'ERR2497994',
 'ERR2498005',
 'ERR2498025',
 'ERR2498028',
 'ERR2498033']
NR = ['ERR2497987',
 'ERR2497988',
 'ERR2497989',
 'ERR2497991',
 'ERR2497995',
 'ERR2497996',
 'ERR2497997',
 'ERR2497998',
 'ERR2497999',
 'ERR2498000',
 'ERR2498001',
 'ERR2498002',
 'ERR2498003',
 'ERR2498004',
 'ERR2498006',
 'ERR2498007',
 'ERR2498008',
 'ERR2498009',
 'ERR2498010',
 'ERR2498013',
 'ERR2498015',
 'ERR2498016',
 'ERR2498019',
 'ERR2498020',
 'ERR2498021',
 'ERR2498023',
 'ERR2498024',
 'ERR2498026',
 'ERR2498027',
 'ERR2498030',
 'ERR2498032']

In [5]:
data = data.loc[STAD_high+STAD_EBV+STAD_MSS+R+NR,:]
grouping = [1]*len(STAD_high)+[0]*len(STAD_EBV+STAD_MSS+R+NR)

In [7]:
X = np.asmatrix(data)
y = np.asarray(grouping)

In [29]:
bc = BaggingClassifierPU(
    SVC(), 
    n_estimators = 1000,  # 1000 trees as usual
    max_samples = sum(y), # Balance the positives and unlabeled in each bag
    n_jobs = -1,
    random_state=100# Use all cores
)

In [30]:
bc.fit(X, y)
res = bc.oob_decision_function_[:,1]

/home/jovi/anaconda3/envs/r4/lib/python3.9/site-packages/sklearn/utils/validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


In [31]:
print(len(STAD_high))
print(len(STAD_EBV))
print(len(STAD_MSS))
print(len(R))
print(len(NR))

40
27
257
7
31


In [32]:
results = pd.DataFrame(dict(zip(STAD_high+STAD_EBV+STAD_MSS+R+NR,res)).items(),columns=["ID","Prob"])
results["subtype"] = ["MSI-H & TMB-H & TILs-H"]*len(STAD_high)+["EBV"]*len(STAD_EBV)+["MSS"]*len(STAD_MSS)+["Responder"]*len(R)+["Non-Responder"]*len(NR)

In [34]:
results.to_csv("TCGA-STAD_Keynote061_PUL_prod.csv",index=False)